## Import Libraries

In [1]:
import numpy as np
import pandas as pd
import scipy.optimize as optimize
import matplotlib.pyplot as plt

In [2]:
# A = Original matrix
# B = Square root of Matrix A
# U, V, L = Matrices obtained after SVD
# Ut, Vt, Lt = Truncated U, V, L matrices
# Bnp = Matrix obtained from U, V, L with no phase added or truncation done
# Bp = Matrix obtained from U, V, L after truncation and phase added
# Bt = Matrix obtained from U, V, L after truncation, no phase added

## Optimization Algorithm

In [3]:
def newMat(x):
    l0,l1 = Lt[0], Lt[1]
    U_new = np.zeros((Ut.shape), dtype=np.cfloat)
    for i in range(Ut.shape[0]):
      r = x[i+2*Ut.shape[0]]
      U_new[i][0] = r*np.cos(x[i]) / l0
      U_new[i][1] = r*(np.sin(x[i]) / l1) * np.exp(1j*x[i+Ut.shape[0]])
    return np.dot(np.dot(U_new,np.diag(Lt)), Vt)


def costFn(x):
    Bp = newMat(x)
    loss = np.linalg.norm(B**2 - np.abs(Bp)**2)
    return (loss)

In [4]:
final_arr = pd.DataFrame(columns=["m", "n", 
                                  "mean_initial_dist", "mean_final_dist", "mean_RI", "std_RI"])

m,n=3,5
r_val = np.zeros((100,3))
print("m = ",m,", n = ",n)
res = np.zeros((100,2))
for i in range(100):
    A = np.random.rand(m, n)
    for j in range(m): A[j] /= sum(A[j])
    B = np.sqrt(A)
    U, L, V = np.linalg.svd(B, full_matrices=False)
    Bnp = np.dot(np.dot(U,np.diag(L)), V)
    Ut = U[:,[0,1]]
    Vt = V[[0,1]]
    Lt = L[[0,1]]
    Bt = np.dot(np.dot(Ut,np.diag(Lt)), Vt)
    initial_guess = np.ones((m*3,), dtype=np.longdouble)
    result = optimize.minimize(fun=costFn, x0=initial_guess, tol=1e-10, method='Nelder-Mead', options={'maxiter':1e+10})
    res[i][0] = (np.linalg.norm(B**2 - Bt**2))
    res[i][1] = costFn(result.x)
    r_val[i] = result.x[-Ut.shape[0]:]
    if ((i+1)%10==0):
      print(i+1, " ", end='')
mean_initial, mean_final = res.mean(axis=0)
ri_mean= ((mean_initial - mean_final)*100/mean_initial)
ri_std = np.mean(np.std(res[:,0]-res[:,1]))
data = {"m": m, "n": n, 
        "mean_initial_dist": mean_initial, "mean_final_dist": mean_final,
        "mean_RI": ri_mean, "std_RI": ri_std}
final_arr = final_arr.append(data, ignore_index=True)
print("\n")

m =  3 , n =  5
10  20  30  40  50  60  70  80  90  100  



In [5]:
r_val

array([[ 1.00316481,  0.95849917,  1.03191106],
       [ 0.99031035,  1.01848289,  0.99615452],
       [ 1.00393011,  1.0020732 ,  0.99558925],
       [ 1.00048449,  1.0012703 ,  1.00248008],
       [ 1.00084465,  0.99760423,  1.00131152],
       [ 0.99480691,  1.00678591,  1.00093363],
       [ 1.04407002,  0.9885927 ,  1.01324873],
       [ 1.02102789,  0.98892811,  0.9806615 ],
       [ 0.99913605,  0.99922316,  1.00140959],
       [ 0.99607838,  1.01593914,  0.98757151],
       [ 1.05098939,  0.99975638,  0.9280609 ],
       [ 1.00785367,  1.00251581,  0.99981157],
       [ 1.01215346,  0.98557958,  0.99852606],
       [ 0.98941124,  1.01262687,  1.00609546],
       [ 1.00012072,  1.00061609,  1.00399492],
       [ 0.99561988,  0.99894035,  1.01404105],
       [ 0.98665677,  0.99906715,  1.02497307],
       [ 1.00082693,  1.0033894 ,  0.99596043],
       [ 0.98632165,  1.00202747,  1.00714777],
       [ 1.00244444,  0.99871498,  0.99932003],
       [ 1.0038918 ,  0.99726885,  1.003

In [6]:
final_arr

,m,n,mean_initial_dist,mean_final_dist,mean_RI,std_RI
0,3.0,5.0,0.157927,0.139243,11.831165,0.02222
